## Constraint detecting model

In this notebook the model to detect constraints is trained.

In previous notebooks the data for this process has been generated.
In this notebook the architecture of the model is defined.
The model is then trained to check if the results are good enough to proceed.

First the prepared data has to be loaded.

The dataset is split into three parts.
The training set containts 100.000 images, the validation set 20.000 and the test set contains the rest of the 119.190 images (19.190).

`src.utils.decode_image_record` can not be used here, because it needs the data to be split before calling it.
Thoughts on the improvement to the data generation process can be read in the conclusion to this first prototype in the report-document.

In [1]:
from datetime import datetime
from os import path

import tensorflow as tf

In [2]:
record_path = path.join('data', 'sep_processed_01', 'crops.tfrecord')

feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}

def decode(example):
    feature = tf.io.parse_single_example(example, feature_description)
    image = tf.io.parse_tensor(feature['image'], tf.float32)
    image.set_shape((96, 96, 1))
    label = feature['label']
    return [image, label]

dataset = tf.data.TFRecordDataset(record_path).shuffle(1000)

def sanitize_dataset(ds):
    autotune = tf.data.experimental.AUTOTUNE
    return (ds
        .map(decode, num_parallel_calls=autotune)    
        .cache()
        .repeat()
        .batch(32)
        .prefetch(buffer_size=autotune))

train = sanitize_dataset(dataset.take(80000))
validate = sanitize_dataset(dataset.skip(80000).take(20000))
test = sanitize_dataset(dataset.skip(100000))

As a baseline the model trained in the `srp` is used.
The only difference is that the input layer expects 96 by 96 images.

This time the model is created using the functional API, which holds no functional differences here.

In [3]:
inputs = tf.keras.Input(shape=(96, 96, 1))
m = tf.keras.layers.Conv2D(16, (4,4), activation='relu', padding='same')(inputs)
m = tf.keras.layers.MaxPooling2D(2,2)(m)
m = tf.keras.layers.Conv2D(32, (4,4), activation='relu', padding='same')(m)
m = tf.keras.layers.MaxPooling2D(2,2)(m)
m = tf.keras.layers.Flatten()(m)
outputs = tf.keras.layers.Dense(3, 'softmax')(m)

model = tf.keras.Model(inputs = inputs, outputs = outputs)

log_dir = path.join('logs', 'sep231', datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
model_path = path.join('models', 'devel', 'crop_detector.h5')

callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, embeddings_freq=1),
    tf.keras.callbacks.ModelCheckpoint(model_path, save_best_only=True)]

model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics   = ['acc'])

history = model.fit(
    train,
    steps_per_epoch=100,
    epochs=60,
    validation_data=validate,
    validation_steps=30)

Train for 100 steps, validate for 30 steps
Epoch 1/60
100/100 [==============================] - 33s 334ms/step - loss: 0.4512 - acc: 0.8459 - val_loss: 0.3423 - val_acc: 0.8573
Epoch 2/60
100/100 [==============================] - 30s 295ms/step - loss: 0.3266 - acc: 0.8656 - val_loss: 0.2914 - val_acc: 0.8823
Epoch 3/60
100/100 [==============================] - 31s 307ms/step - loss: 0.2720 - acc: 0.8953 - val_loss: 0.2298 - val_acc: 0.9094
Epoch 4/60
100/100 [==============================] - 31s 314ms/step - loss: 0.2261 - acc: 0.9106 - val_loss: 0.2037 - val_acc: 0.9260
Epoch 5/60
100/100 [==============================] - 36s 364ms/step - loss: 0.2285 - acc: 0.9078 - val_loss: 0.2176 - val_acc: 0.9177
Epoch 6/60
100/100 [==============================] - 43s 434ms/step - loss: 0.1869 - acc: 0.9281 - val_loss: 0.1778 - val_acc: 0.9302
Epoch 7/60
100/100 [==============================] - 48s 480ms/step - loss: 0.1869 - acc: 0.9319 - val_loss: 0.1718 - val_acc: 0.9438
Epoch 8/60
1

KeyboardInterrupt: 

In [4]:
print(model.evaluate(test, steps=100))

100/100 [==============================] - 87s 869ms/step - loss: 0.1104 - acc: 0.9594
[0.11041362623684109, 0.959375]
